In [ ]:
import pandas as pd
import requests

In [ ]:
df_system_account = pd.read_excel(r"C:\Users\Administrator\Downloads/system_account_10180.xlsx")
df_items = pd.read_excel(r"C:\Users\Administrator\Downloads/item_position_10180.xlsx")

In [ ]:
df_system_account.drop_duplicates(subset=['system_account_name'], inplace=True)

In [ ]:
df_system_account['system_account_name'].unique()

In [ ]:
url_api = "http://srv-caito-prod02.cisbox.com:8099/llm-accounting-mapping"

In [ ]:
system_account_name_list = df_system_account['system_account_name'].unique().tolist()

In [ ]:
from tqdm import tqdm

count = 0

for idx, row in tqdm(df_items.iterrows(), total=df_items.shape[0]):
    # Only predict if system_account_name_phi4 is NaN (not yet predicted)
    if pd.isna(row.get('system_account_name_phi4')):
        item_name = row['item_name']
        payload = {
            "item_name": item_name,
            "account_name_list": str(system_account_name_list)
        }

        response = requests.post(url_api, json=payload)

        if response.status_code == 200:
            # Extract the result from response
            result = response.text

            # Assign it back to the dataframe
            df_items.loc[idx, 'system_account_name_phi4'] = result

            # Increase the count
            count += 1

            # Save the file every 100 predictions
            if count % 10000000 == 0:
                df_items.to_excel(r"C:\Users\Administrator\Downloads\item_position_10180_phi4.xlsx", index=False)
                print(f"Saved after {count} new predictions.")

# --- After loop completes, do a final save in case there are leftover predictions ---
df_items.to_excel(r"C:\Users\Administrator\Downloads\item_position_10180_phi4.xlsx", index=False)
print("Final save complete.")